In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# final_ems_data_slice columns
columns = [
    "index", "ID", "Hospital_Code", "Date",
    "APOT", "Impression", "Postal_Code", "AgencyNumber",
    "Agency_Unit", "Lat", "Long", "Status",
    
]

target = ["Status"]

In [6]:
# Load the data
file_path = Path('final_ems_data_slice_test.csv')
df = pd.read_csv(file_path, encoding='utf-8')

df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

df = df.drop(["APOT"], axis=1)

df.head()



,index,ID,Hospital_Code,Date,Impression,Postal_Code,AgencyNumber,Agency_Unit,Lat,Long,Status
0,0,776,508,7/19/2017,T14.90,95820,92905,92905-M22,38.554938,-121.456751,0 (< 20 min)
1,1,824,508,8/20/2017,G89.1,95831,92905,92905-M17,38.554938,-121.456751,0 (< 20 min)
2,2,854,197,9/8/2017,T14.90,95608,92905,92905-M17,38.601860,-121.391908,0 (< 20 min)
3,3,867,475,9/14/2017,R53.1,95624,92905,92905-M23,38.570890,-121.469532,1 (21 - 60 min)
4,4,951,475,10/19/2017,R10.84,95816,92905,92905-M17,38.570890,-121.469532,0 (< 20 min)


In [7]:
# get data types
df.dtypes

index              int64
ID                 int64
Hospital_Code      int64
Date              object
Impression        object
Postal_Code       object
AgencyNumber       int64
Agency_Unit       object
Lat              float64
Long             float64
Status            object
dtype: object

In [8]:
# Convert the target column values to low_risk and high_risk based on their values
x = {'0 (< 20 min)': 'benchmark'}   
df = df.replace(x)

x = dict.fromkeys(['1 (21 - 60 min)', '2 (61 - 120 min)', '3 (121 - 180 min)', '4 (181 + min)'], 'extreme')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,index,ID,Hospital_Code,Date,Impression,Postal_Code,AgencyNumber,Agency_Unit,Lat,Long,Status
0,0,776,508,7/19/2017,T14.90,95820,92905,92905-M22,38.554938,-121.456751,benchmark
1,1,824,508,8/20/2017,G89.1,95831,92905,92905-M17,38.554938,-121.456751,benchmark
2,2,854,197,9/8/2017,T14.90,95608,92905,92905-M17,38.601860,-121.391908,benchmark
3,3,867,475,9/14/2017,R53.1,95624,92905,92905-M23,38.570890,-121.469532,extreme
4,4,951,475,10/19/2017,R10.84,95816,92905,92905-M17,38.570890,-121.469532,benchmark


# Split the Data into Training and Testing

In [9]:
# Create our features
X = df.drop(columns='Status')
X = pd.get_dummies(X)

# Create our target
y = df[target]

In [10]:
X.describe()

,index,ID,Hospital_Code,AgencyNumber,Lat,Long,Date_1/1/2018,Date_1/10/2018,Date_1/11/2018,Date_1/12/2018,...,Agency_Unit_605-S-753,Agency_Unit_605-S-754,Agency_Unit_605-S-755,Agency_Unit_92905-M102,Agency_Unit_92905-M17,Agency_Unit_92905-M2,Agency_Unit_92905-M21,Agency_Unit_92905-M22,Agency_Unit_92905-M23,Agency_Unit_92905-M3
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,101355.360000,361.748000,7640.800000,223.159364,-119.991811,0.001000,0.005000,0.005000,0.002000,...,0.066000,0.006000,0.050000,0.009000,0.026000,0.002000,0.013000,0.002000,0.021000,0.003000
std,288.819436,44261.754278,147.587345,24466.173628,1937.984056,15.118604,0.031623,0.070569,0.070569,0.044699,...,0.248406,0.077266,0.218054,0.094488,0.159215,0.044699,0.113331,0.044699,0.143456,0.054717
min,0.000000,776.000000,5.000000,605.000000,38.463174,-121.469532,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,249.750000,71723.250000,280.000000,605.000000,38.554938,-121.469532,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,499.500000,106003.500000,475.000000,605.000000,38.570890,-121.456751,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,749.250000,143083.250000,475.000000,605.000000,38.570890,-121.416364,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,999.000000,152172.000000,549.000000,92905.000000,20549.000000,38.573060,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
# Check the balance of our target values
y['Status'].value_counts()

extreme      533
benchmark    467
Name: Status, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.shape

(750, 465)

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)



BalancedRandomForestClassifier(random_state=1)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6613171888013325

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


array([[85, 36],
       [49, 80]], dtype=int64)

In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  benchmark       0.63      0.70      0.62      0.67      0.66      0.44       121
    extreme       0.69      0.62      0.70      0.65      0.66      0.43       129

avg / total       0.66      0.66      0.66      0.66      0.66      0.44       250



In [18]:
# List the features sorted in descending order by feature importance
importances = model.feature_importances_
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.087979237536251, 'ID'),
 (0.08711941544642246, 'index'),
 (0.056049483608921875, 'Long'),
 (0.05252486445134407, 'Hospital_Code'),
 (0.045172374054934815, 'Lat'),
 (0.015320413610195115, 'Impression_R53.1'),
 (0.013335274178444252, 'Agency_Unit_605-S-746'),
 (0.012297104583841473, 'Postal_Code_95758'),
 (0.012158629306274791, 'Agency_Unit_605-S-751'),
 (0.011064390686369987, 'Agency_Unit_605-S-742'),
 (0.010283260949513357, 'Impression_T14.90'),
 (0.010115499852852206, 'Impression_F99'),
 (0.010017713253176281, 'Postal_Code_95816'),
 (0.009937618164836676, 'Impression_R10.84'),
 (0.00913440237092876, 'Postal_Code_95838'),
 (0.00769132144598849, 'Postal_Code_95815'),
 (0.007688590512035068, 'Impression_I20.9'),
 (0.007315783388109334, 'Impression_G89.1'),
 (0.0072997067957971595, 'Agency_Unit_605-S-755'),
 (0.0071621763073143295, 'Agency_Unit_605-S-753'),
 (0.006957183119315394, 'Impression_R51'),
 (0.006954447244169347, 'Impression_J80'),
 (0.006685010576859546, 'Postal_Code_95823'

### Easy Ensemble AdaBoost Classifier

In [19]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [20]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6303094368633481

In [21]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[85, 36],
       [57, 72]], dtype=int64)

In [22]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  benchmark       0.60      0.70      0.56      0.65      0.63      0.40       121
    extreme       0.67      0.56      0.70      0.61      0.63      0.39       129

avg / total       0.63      0.63      0.63      0.63      0.63      0.39       250

